In [3]:
import requests
url = "https://github.com/LCAV/pyroomacoustics/blob/master/examples/data/INRIA_MUSIS.stl"
mesh_url = "../tmp/INRIA_MUSIS.stl"

from pathlib import Path
if not Path(mesh_url).is_file():
    print(f"download data to {mesh_url}")
    r = requests.get(url, allow_redirects=True)
    open(mesh_url, 'wb').write(r.content)